# Preprocessing
link - https://www.kaggle.com/code/amirhosseinzinati/adult-income-k-nearest-neighbors-knn (REMOVE BEFORE SUBMIT)

### install & import

In [16]:
!pip3 install pandas
!pip3 install numpy
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 724.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 1.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 1.9 MB/s eta 0:00:0000:0100:01


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn import preprocessing
warnings.filterwarnings("ignore")

### Having a sneak pick

In [24]:
def preprocess_adult():
    """
    Preprocesses the adult dataset by performing the following steps:
    1. Reads the dataset from the file './datasets/adult.csv'.
    2. Replaces any '?' values in the 'workclass', 'occupation', and 'native-country' columns with NaN.
    3. Drops any rows with missing values.
    4. Drops duplicate rows.
    5. Drops the 'educational-num', 'capital-gain', and 'capital-loss' columns.
    6. Encodes categorical columns using LabelEncoder.
    7. Returns the preprocessed dataframe.

    Returns:
    - df1 (pandas.DataFrame): Preprocessed dataframe.
    """
    data = pd.read_csv("./datasets/adult.csv")
    data.isin(['?']).sum()
    data['workclass']=data['workclass'].replace('?',np.nan)
    data['occupation']=data['occupation'].replace('?',np.nan)
    data['native-country']=data['native-country'].replace('?',np.nan)
    df = data.copy()
    df.dropna(how='any',inplace=True)
    df = df.drop_duplicates()
    df1 = df.drop(['educational-num', 'capital-gain', 'capital-loss'], axis=1)

    label_encoder = preprocessing.LabelEncoder()

    df1['gender'] = label_encoder.fit_transform(df1['gender'])
    df1['workclass'] = label_encoder.fit_transform(df1['workclass'])
    df1['education'] = label_encoder.fit_transform(df1['education'])
    df1['marital-status'] = label_encoder.fit_transform(df1['marital-status'])
    df1['occupation'] = label_encoder.fit_transform(df1['occupation'])
    df1['relationship'] = label_encoder.fit_transform(df1['relationship'])
    df1['race'] = label_encoder.fit_transform(df1['race'])
    df1['native-country'] = label_encoder.fit_transform(df1['native-country'])
    df1['income'] = label_encoder.fit_transform(df1['income'])

    return df1

In [25]:
preprocess_adult()

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,gender,hours-per-week,native-country,income
0,25,2,226802,1,4,6,3,2,1,40,38,0
1,38,2,89814,11,2,4,0,4,1,50,38,0
2,28,1,336951,7,2,10,0,4,1,40,38,1
3,44,2,160323,15,2,6,0,2,1,40,38,1
5,34,2,198693,0,4,7,1,4,1,30,38,0
...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,2,257302,7,2,12,5,4,0,38,38,0
48838,40,2,154374,11,2,6,0,4,1,40,38,1
48839,58,2,151910,11,6,0,4,4,0,40,38,0
48840,22,2,201490,11,4,0,3,4,1,20,38,0
